In [ ]:
from lib.model_test import D2Net
from lib.utils import preprocess_image
from lib.pyramid import process_multiscale

import numpy as np
import torch

import scipy
import scipy.io
import scipy.misc

import imageio

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
##settings 
model_file = '../../data/teacher_models/d2net/d2_ots.pth'
use_relu = True
use_cuda = torch.cuda.is_available()
max_edge = 1600
max_sum_edges = 2800
preprocessing = 'caffe'
multiscale=False
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = D2Net(
    model_file=model_file,
    use_relu=use_relu,
    use_cuda=use_cuda
)

In [ ]:
image = imageio.imread('../../data/AachenDayNight/images_upright/db/1.jpg')
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
resized_image = image
if max(resized_image.shape) > max_edge:
    resized_image = scipy.misc.imresize(
            resized_image,
            max_edge / max(resized_image.shape)
        ).astype('float')
if sum(resized_image.shape[: 2]) > max_sum_edges:
    resized_image = scipy.misc.imresize(
            resized_image,
            max_sum_edges / sum(resized_image.shape[: 2])
        ).astype('float')

In [ ]:
fact_i = image.shape[0] / resized_image.shape[0]
fact_j = image.shape[1] / resized_image.shape[1]
input_image = preprocess_image(
        resized_image,
        preprocessing=preprocessing
    )

In [ ]:
with torch.no_grad():
    if multiscale:
        keypoints, scores, descriptors = process_multiscale(
            torch.tensor(
                input_image[np.newaxis, :, :, :].astype(np.float32),
                device=device
            ),
            model
        )
    else:
        keypoints, scores, descriptors = process_multiscale(
            torch.tensor(
                input_image[np.newaxis, :, :, :].astype(np.float32),
                device=device
            ),
            model,
            scales=[1]
        )

In [ ]:
# Input image coordinates
keypoints[:, 0] *= fact_i
keypoints[:, 1] *= fact_j
# i, j -> u, v
keypoints = keypoints[:, [1, 0, 2]]

In [ ]:
print(keypoints.shape)
print(image.shape)
print(keypoints[:, 0:2].max(axis=0))
print(descriptors.shape)